In [ ]:
#!pipenv install pandas plotly scikit-learn optuna ipywidgets ipykernel nbformat matplotlib pingouin gradio

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import optuna